In [5]:
import tensorflow as tf
import numpy as np
import os
import pandas as pd
import csv
from resizeimage import resizeimage
from PIL import Image
from tensorflow.python.ops.image_ops_impl import ResizeMethod  
#from pandas_ml import ConfusionMatrix
from random import shuffle
import datetime


In [7]:
a=[]
for i in range(0,200000,2):
    a.append(0)
    a.append(1)
bl=np.zeros((200000,2))
bl[np.arange(200000), a]=1

In [8]:
a=[]
for i in range(0,100000,2):
    a.append(0)
    a.append(1)
    
test_label=np.zeros((100000,2))
test_label[np.arange(100000), a]=1

In [9]:
def weight_variable(shape,na):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial,name=na)

def bias_variable(shape,na):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=na)

def conv2d(xe, W):
    return tf.nn.conv2d(xe, W, strides=[ 1, 1, 1, 1], padding='SAME')

def conv1d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 2, 2, 1], padding='VALID')


def max_pool(xe):
    return tf.nn.max_pool(xe, ksize=[1, 3, 3, 1],strides=[1,2,2, 1], padding='SAME')

def max_pool_s2(xe):
    return tf.nn.max_pool(xe, ksize=[1, 3,3, 1],strides=[1, 2, 2, 1], padding='SAME')

In [10]:
def bn(is_training,inputs,decay=0.999):
    scale=tf.Variable(tf.ones(inputs.get_shape()[-1:]))
    beta = tf.Variable(tf.zeros(inputs.get_shape()[-1:]))
    pop_mean = tf.Variable(tf.zeros(inputs.get_shape()[-1:]), trainable=False)
    pop_var = tf.Variable(tf.ones(inputs.get_shape()[-1:]), trainable=False)
    mean = tf.cond(is_training,lambda:tf.nn.moments(inputs,[0,1,2])[0],lambda:tf.ones(tf.shape(inputs.get_shape()[-1:]))*pop_mean)
    var = tf.cond(is_training,lambda:tf.nn.moments(inputs,[0,1,2])[1],lambda:tf.ones(tf.shape(inputs.get_shape()[-1:]))*pop_var)
    train_mean = tf.cond(is_training,lambda:tf.assign(pop_mean,pop_mean * decay + mean * (1 - decay)),lambda:tf.ones([0]))
    train_var= tf.cond(is_training,lambda:tf.assign(pop_var,pop_var * decay + var * (1 - decay)),lambda:tf.ones([0]))
    with tf.control_dependencies([train_mean,train_var]):
        return tf.nn.batch_normalization(inputs,mean,var,beta,scale,epsilon)

In [11]:
tf.reset_default_graph()

xin = tf.placeholder(tf.float32,shape=[None, 32,32,3])
y_ = tf.placeholder(tf.float32,shape=[None, 2])
epsilon=0.000001
is_training=tf.placeholder(tf.bool,[])

x_image = tf.reshape(xin, [-1, 32,32, 3])

W1=weight_variable([2,2,3,32],'w1')
b1=bias_variable([32],'b2')

h_conv1=tf.nn.relu(bn(is_training,conv2d(x_image,W1))+b1)


w2=weight_variable([2,2,32,32],'w2')
b2=bias_variable([32],'b2')
conv1=tf.nn.relu(bn(is_training,conv2d(h_conv1,w2))+b2)
conv1=max_pool(conv1)

w3=weight_variable([2,2,32,64],'w3')
b3=bias_variable([64],'b3')
conv2=tf.nn.relu(bn(is_training,conv2d(conv1,w3))+b3)

w4=weight_variable([2,2,64,64],'w4')
b4=bias_variable([64],'b4')
conv3=tf.nn.relu(bn(is_training,conv2d(conv2,w4))+b4)
conv3=max_pool(conv3)


w5=weight_variable([2,2,64,128],'w5')
b5=bias_variable([128],'b5')
conv4=tf.nn.relu(bn(is_training,conv2d(conv3,w5))+b5)

w6=weight_variable([2,2,128,128],'w6')
b6=bias_variable([128],'b6')
conv5=tf.nn.relu(bn(is_training,conv2d(conv4,w6))+b6)
conv5=max_pool(conv5)

w7=weight_variable([2,2,128,256],'w7')
b7=bias_variable([256],'b7')
conv6=tf.nn.relu(bn(is_training,conv2d(conv5,w7))+b7)

w8=weight_variable([2,2,256,256],'w8')
b8=bias_variable([256],'b8')
conv7=tf.nn.relu(bn(is_training,conv2d(conv6,w8))+b8)
#conv6=max_pool(conv6)

W_fc1=weight_variable([16*256,1000],'w_fc1')
b_fc1=bias_variable([1000],'b_fc1')
y1=tf.reshape(conv6,[-1,16*256])
y_fc1=tf.nn.relu(tf.matmul(y1,W_fc1)+b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(y_fc1, keep_prob)

W_fc2=weight_variable([1000,100],'w_fc2')
b_fc2=bias_variable([100],'b_fc2')
y_fc2=tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

h_fc2_drop = tf.nn.dropout(y_fc2, keep_prob)

W_fc3=weight_variable([100,2],'w_fc3')
b_fc3=bias_variable([2],'b_fc3')
y_fc3=tf.nn.relu(tf.matmul(h_fc2_drop,W_fc3)+b_fc3)


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [12]:
pred = tf.argmax(y_fc3, 1)
lab = tf.argmax(y_, 1)

In [13]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_fc3))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.cast(tf.equal(tf.argmax(y_fc3, 1), tf.argmax(y_, 1)),tf.float32)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
r=os.listdir(r'F:/A. Segmentation/1. Original Images/Train_resized')
r.sort()
qt={}
for i in range(0,len(r),3): 
    qt.update({r[i]:bl[i]})
    qt.update({r[i+1]:bl[i+1]})
    qt.update({r[i+2]:bl[i+2]})
    
ri=os.listdir(r'F:/A. Segmentation/1. Original Images/Test_resized')
ri.sort()
qte={}
for i in range(0,len(ri),3): 
    qte.update({ri[i]:test_label[i]})
    qte.update({ri[i+1]:test_label[i+1]})
    qte.update({ri[i+2]:test_label[i+2]})

In [15]:
saver = tf.train.Saver(max_to_keep=10000)

In [ ]:
with tf.Session() as sess:
    print('session started')
    sess.run(tf.global_variables_initializer())
    saver.restore(sess,'F:/Hard-Exudate-Detection-in-Retinal-Fundus-Images-Using-Deep-Learning-master/runs/model_100.ckpt')
    r=os.listdir(r'F:/A. Segmentation/1. Original Images/Train_resized')
    r.sort()
    shuffle(r)
    no_test=os.listdir(r'F:/A. Segmentation/1. Original Images/Test_resized')
    no_test.sort()
    a=datetime.datetime.now().time()
    print(a)
    for epoch in range(200):
        a=datetime.datetime.now().time()
        print('EPOCH %d started at %s'%(epoch+1,a))
        for i in range(0,54,9):
            r_lis=r[i:i+9]
            for k in r_lis: 
                a=Image.open(r'F:/A. Segmentation/1. Original Images/Train_resized/%s'%k)
                npg=np.array(a,dtype='f')
                nt=np.array([npg])
                s=np.array([qt[k]])
                if k==r_lis[0]:
                    ti=np.array([npg])
                    labels=np.array([qt[k]])
                else:
                    ti = np.append(ti,nt,axis=0)#300
                    labels = np.append(labels,s,axis=0) 
            train_step.run(feed_dict={xin:ti,y_:labels,is_training:True,keep_prob: 0.5})
            pre=correct_prediction.eval(feed_dict={xin:ti,y_:labels,is_training:False,keep_prob: 1})
            if i==0:
                axx=pre
            else:
                axx=np.append(axx,pre)
        train=sum(axx)/len(axx)
        print('train accuracy of EPOCH %d is: %f'%((epoch+1),train))
        save_path=saver.save(sess,r'F:/Hard-Exudate-Detection-in-Retinal-Fundus-Images-Using-Deep-Learning-master/runs/model_%03d.ckpt'%(epoch+1))        
        
        for il in range(0,27,9):
            test_lis=no_test[il:il+9]
            for k in test_lis: 
                a=Image.open(r'F:/A. Segmentation/1. Original Images/Test_resized/%s'%k)
                npg=np.array(a,dtype='f')
                nt=np.array([npg])
                s=np.array([qte[k]])
                if k==test_lis[0]:
                    ti=np.array([npg])
                    labels=np.array([qte[k]])
                else:
                    ti = np.append(ti,nt,axis=0)#300
                    labels = np.append(labels,s,axis=0) 
            pre=correct_prediction.eval(feed_dict={xin:ti,y_:labels,is_training:False,keep_prob: 1})
            if il==0:
                axx=pre
            else:
                axx=np.append(axx,pre)
        test=sum(axx)/len(axx)
        print('test accuracy of EPOCH %d is: %f'%((epoch+1),test))
        if test>0.6:
            pr=np.array([])
            ded=np.array([])
            for il in range(0,27,9):
                test_lis=no_test[il:il+9]
                for k in test_lis: 
                    a=Image.open(r'F:/A. Segmentation/1. Original Images/Test_resized/%s'%k)
                    npg=np.array(a,dtype='f')
                    nt=np.array([npg])
                    s=np.array([qte[k]])
                    if k==test_lis[0]:
                        test_i=np.array([npg])
                        test_la=np.array([qte[k]])
                    else:
                        test_i = np.append(test_i,nt,axis=0)#300
                        test_la = np.append(test_la,s,axis=0)
                pre=pred.eval(feed_dict={xin:test_i,y_:test_la,is_training:False,keep_prob: 1})
                pr=np.append(pr,pre,axis=0)
                labe=lab.eval(feed_dict={xin:test_i,y_:test_la,is_training:False,keep_prob: 1})
                ded=np.append(ded,labe,axis=0)
            np.savez(r'F:/Hard-Exudate-Detection-in-Retinal-Fundus-Images-Using-Deep-Learning-master/runs/roc/label_%02d.npz'%epoch , ded )
            np.savez(r'F:/Hard-Exudate-Detection-in-Retinal-Fundus-Images-Using-Deep-Learning-master/runs/roc/predi_%02d.npz'%epoch , pr )
        a=datetime.datetime.now().time()
        print('EPOCH %d ENDED at %s'%(epoch+1,a))

session started
INFO:tensorflow:Restoring parameters from F:/Hard-Exudate-Detection-in-Retinal-Fundus-Images-Using-Deep-Learning-master/runs/model_100.ckpt
01:51:36.885512
EPOCH 1 started at 01:51:36.885512
train accuracy of EPOCH 1 is: 0.500000
test accuracy of EPOCH 1 is: 0.518519
EPOCH 1 ENDED at 01:51:39.169131
EPOCH 2 started at 01:51:39.169131
train accuracy of EPOCH 2 is: 0.500000
test accuracy of EPOCH 2 is: 0.518519
EPOCH 2 ENDED at 01:51:40.728171
EPOCH 3 started at 01:51:40.728171
train accuracy of EPOCH 3 is: 0.500000
test accuracy of EPOCH 3 is: 0.518519
EPOCH 3 ENDED at 01:51:42.297329
EPOCH 4 started at 01:51:42.297329
train accuracy of EPOCH 4 is: 0.500000
test accuracy of EPOCH 4 is: 0.518519
EPOCH 4 ENDED at 01:51:43.965051
EPOCH 5 started at 01:51:43.965051
train accuracy of EPOCH 5 is: 0.500000
test accuracy of EPOCH 5 is: 0.518519
EPOCH 5 ENDED at 01:51:45.614502
EPOCH 6 started at 01:51:45.614502
train accuracy of EPOCH 6 is: 0.500000
test accuracy of EPOCH 6 is: 

In [3]:
from PIL import Image
import os

# Define the source and target directories
source_dir = 'F:/A. Segmentation/1. Original Images/a. Training Set'
target_dir = 'F:/A. Segmentation/1. Original Images/Train_resized'

# Create target directory if it doesn't exist
if not os.path.exists(target_dir):
    os.makedirs(target_dir)

# Loop through all files in the source directory
for filename in os.listdir(source_dir):
    file_path = os.path.join(source_dir, filename)
    
    # Check if it's a file (to avoid directories or non-image files)
    if os.path.isfile(file_path):
        try:
            # Open the image
            with Image.open(file_path) as img:
                # Resize the image to 32x32
                resized_img = img.resize((32, 32))
                
                # Define the new file path
                new_file_path = os.path.join(target_dir, filename)
                
                # Save the resized image
                resized_img.save(new_file_path)
                
                print(f"Resized and saved: {filename}")
        except Exception as e:
            print(f"Error processing {filename}: {e}")


Resized and saved: IDRiD_01.jpg
Resized and saved: IDRiD_02.jpg
Resized and saved: IDRiD_03.jpg
Resized and saved: IDRiD_04.jpg
Resized and saved: IDRiD_05.jpg
Resized and saved: IDRiD_06.jpg
Resized and saved: IDRiD_07.jpg
Resized and saved: IDRiD_08.jpg
Resized and saved: IDRiD_09.jpg
Resized and saved: IDRiD_10.jpg
Resized and saved: IDRiD_11.jpg
Resized and saved: IDRiD_12.jpg
Resized and saved: IDRiD_13.jpg
Resized and saved: IDRiD_14.jpg
Resized and saved: IDRiD_15.jpg
Resized and saved: IDRiD_16.jpg
Resized and saved: IDRiD_17.jpg
Resized and saved: IDRiD_18.jpg
Resized and saved: IDRiD_19.jpg
Resized and saved: IDRiD_20.jpg
Resized and saved: IDRiD_21.jpg
Resized and saved: IDRiD_22.jpg
Resized and saved: IDRiD_23.jpg
Resized and saved: IDRiD_24.jpg
Resized and saved: IDRiD_25.jpg
Resized and saved: IDRiD_26.jpg
Resized and saved: IDRiD_27.jpg
Resized and saved: IDRiD_28.jpg
Resized and saved: IDRiD_29.jpg
Resized and saved: IDRiD_30.jpg
Resized and saved: IDRiD_31.jpg
Resized 